In [27]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np

class Actividad():
    '''Representa una de las actividades de la columna "Actividad" del
    documento de tipo .xlsx proporcionado.'''
    def __init__(self, nombre, descripcion, duracion, precedentes):
        '''Inicializa una de las actividades del documento con distintos
        atributos por definir.'''
        self.nombre = nombre # Numero de cada actividad
        self.descripcion = descripcion # Descripcion de cada actividad
        self.duracion = duracion # Duracion de cada actividad
        self.precedentes = precedentes
        # Actividades precedentes de cada actividad

        self.sucesores = []
        # Lista donde se almacenan las actividades sucesoras de cada actividad

class Simulacion():
    '''Representa la simulacion de la elaboracion del proyecto, con la
    grafica/red generada del docuemnto proporcionado.'''
    def __init__(self, actividades, nodo_inicial, nodo_final):
        '''Inicializa la simulacion de la elaboracion del proyecto'''
        self.actividades = actividades
        # Diccionario de las actividades del proyecto

        self.nodo_inicial = nodo_inicial
        # Establece el nodo inical de la grafica/red del proyecto

        self.nodo_final = nodo_final
        # Establece el nodo final de la grafica/red del proyecto

    def fechas(self):
      '''Fecha mas proxima y la fecha mas lejana de cada actividad del
      proyecto'''
      fechas_mas_proximas = {actividad.nombre: 0 for actividad in self.actividades.values()}
      # Diccionario para almacenar las actividades mas proximas de cada
      # actividad.

      fechas_mas_lejanas = {actividad.nombre: 0 for actividad in self.actividades.values()}
      # Diccionario para almacenar las fechas mas lejanas de cada actividad.

      def calcular_fechas(nodo, tiempo_acumulado):
        '''Calcula la fecha mas proxima y la fecha mas lejana de cada
        actividad del proyecto'''
        fechas_mas_proximas[nodo] = max(fechas_mas_proximas[nodo], tiempo_acumulado)
        # Calcula la fecha mas proxima de la actividad.

        tiempo_acumulado += self.actividades[nodo].duracion

        fechas_mas_lejanas[nodo] = max(fechas_mas_lejanas[nodo], tiempo_acumulado)
        # Calcula la fecha mas lejana de la actividad.

        # Recorre los sucesores de la actividad en el nodo y manda a
        # llamar a la funcion para calcular sus fechas.
        for sucesor in self.actividades[nodo].sucesores:
                calcular_fechas(sucesor.nombre, tiempo_acumulado)

      # Manda a llamar a la funcion para calcular fechas desde la primera
      # actividad en el nodo inicial.
      calcular_fechas(self.nodo_inicial, 0)

      # Devuelve las fechas mas proximas y mas lejanas.
      return fechas_mas_proximas, fechas_mas_lejanas

    def generar_reporte(self, fechas_mas_proximas, fechas_mas_lejanas):
      '''Crea un reporte del proyecto que incluye el tiempo minimo para la
      realizacion de este, asi como sus actividades criticas.'''

      # Calcula el tiempo minimo necesario para la elaboracion del proyecto.
      tiempo_minimo = fechas_mas_lejanas[self.nodo_final]

      # Identifica las actividades criticas del proyecto
      # (aquellas cuya fecha mas proxima es igual a su fecha mas lejana).
      actividades_criticas = [actividad.nombre for actividad in self.actividades.values() if fechas_mas_proximas[actividad.nombre] == fechas_mas_lejanas[actividad.nombre] - actividad.duracion]

      #Escribe el reporte del proyecto de tipo archivo .txt incluyendo el
      # tiempo minimo requerido para realizar el proyecto, asi como sus
      # actividades criticas.
      with open('reporte.txt', 'w', encoding='utf-8') as file:
            file.write(f"El tiempo minimo requerido para la elaboración del proyecto son {tiempo_minimo} semanas.\n\n")
            file.write("Actividades críticas:\n\n")
            for nombre_actividad in actividades_criticas:
                file.write(f"{nombre_actividad}. {self.actividades[nombre_actividad].descripcion}\n")

def leer_archivo(archivo):
  '''Lee un archivo de tipo .xlsx que contiene la informacion sobre las
  especificaciones del proyecto.'''
  df = pd.read_excel(archivo)
  # Lee el archivo de excel y crea un data frame de este.

  actividades = {}
  # Diccionario para almacenar las actividades del proyecto
  # obtenidas del documento de excel.

  # Itera sobre los elementos del data frame para obtener los atributos de las
  # actividades.
  for i, fila in df.iterrows():
        nombre = fila['Actividad']
        descripcion = fila['Descripcion']
        duracion = fila['Duracion']
        precedentes = [] if pd.isna(fila['Precedentes']) or fila['Precedentes'] == '-' else [int(p) for p in str(fila['Precedentes']).split(',') if p]
        # Revisa si alguno de los precedentes de las actividades es de tipo
        # nan o no tiene precedentes.
        # Los convierte a una lista de enteros.

        actividades[nombre] = Actividad(nombre, descripcion, duracion, precedentes)
  return actividades
  # Devuelve el diccionario de las actividades del proyecto.

def obtener_reporte():
    '''Lee el contenido del archivo de reporte y lo devuelve como una cadena.'''
    with open('reporte.txt', 'r', encoding='utf-8') as file:
        contenido = file.read()
    return contenido


def main():
    archivo_excel = '/content/drive/MyDrive/Creacion_de_la_carrera_de_mat_ap.xlsx'
    # Establece el archivo a partir del cual se obtendra la informacion.

    # Se obtienen las actividades, asi como el nodo inicial y el nodo final al
    # leer el documento definido previamente.
    actividades = leer_archivo(archivo_excel)
    nodo_inicial = next(iter(actividades.keys()))
    nodos_sin_sucesores = set(actividades.keys())

    # Itera sobre las actividades y los precedentes para crear la grafica/red
    # del proyecto.
    for actividad in actividades.values():
        for precedente in actividad.precedentes:
            actividad.sucesores.append(actividades[precedente])
            nodos_sin_sucesores.discard(actividad.nombre)

    # Verifica si hay mas de un nodo sin sucesores y en caso de ser asi,
    # lo indica al usuario al imprimir el mensaje.
    if len(nodos_sin_sucesores) != 1:
        raise ValueError("La red es incorrecta, solo debe haber un nodo sin sucesores.")

    nodo_final = next(iter(nodos_sin_sucesores))

    simulacion = Simulacion(actividades, nodo_inicial, nodo_final)
    #Inicia la simulacion y construye la grafica/red.

    fechas_mas_proximas, fechas_mas_lejanas = simulacion.fechas()
    # Calcula las fechas mas proximas y mas lejanas de las actividades.

    simulacion.generar_reporte(fechas_mas_proximas, fechas_mas_lejanas)
    # Genera el reporte del proyecto.

    # Imprime el reporte.
    reporte = obtener_reporte()
    print(reporte)

if __name__ == '__main__':
    main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
El tiempo minimo requerido para la elaboración del proyecto son 3 semanas.

Actividades críticas:

1. Decision preliminar



In [25]:
# Este codigo es de como modele la grafica/red de la forma en que Jose nos
# enseño en clase.

dic = {'A':[], 'B':['A'], 'C':['B'], 'D':['C'], 'E':['D'], 'F':['D'], 'G':['D'], 'H':['D'], 'I':['D'], 'J':['E'], 'K':['F'], 'L':['G'], 'M':['H'], 'N':['K'], 'O':['J','N'],'R':['O'],'S':['R'], 'P':['L','M','R'], 'Q':['I','P'], 'T':['Q','S'], 'U':['T']}


sub = {}
arcs = {}

for a, ps in dic.items():
    for p in ps:
        if p not in sub:
            sub[p] = []
        sub[p].append(a)


nodes = [0,1]
done = [[],[]]
ind = 2
arrows = []
arrows = []
arcs = {}

for k,v in dic.items():
  if k not in sub:
        sub[k] = []
  #print(k)
  if v == []:
    #print(k)
    nodes.append(ind)
    arrows.append((0,ind))
    done.append([k])
    arcs[k] = (0,ind)
    ind +=1
  elif not v in done:
    separated = [[elem] for elem in v]
    for elem in separated:
      arrows.append((done.index(elem),ind))
    done.append(v)
    nodes.append(ind)
    ind +=1
    if sub[k] == []:
      arrows.append((done.index(v),1))
      arcs[k] = (done.index(v),1)
    else:
      nodes.append(ind)
      arrows.append((done.index(v),ind))
      done.append([k])
      arcs[k] = (done.index(v),ind)
      ind +=1
  else:
    if sub[k] == []:
      arrows.append((done.index(v),1))
      arcs[k] = (done.index(v),1)
    else:
      nodes.append(ind)
      arrows.append((done.index(v),ind))
      done.append([k])
      arcs[k] = (done.index(v),ind)
      ind +=1

#print(nodes)
#print(arrows)
#print(done)
#print(arcs)